In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

/home/ken/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['"', 'Rich', 'in', 'Love', '"', 'is', 'a', 'slice', '-', 'of', '-', 'life', 'film', 'which', 'takes', 'the', 'viewer', 'into', 'the', 'goings', 'on', 'of', 'a', 'somewhat', 'quirky', 'Charleston', ',', 'SC', 'family', '.', 'Highly', 'romanticized', ',', 'beautifully', 'shot', ',', 'well', 'written', 'and', 'acted', ',', '"', 'RIL', '"', 'washes', 'over', 'you', 'like', 'a', 'summer', 'breeze', 'as', 'its', 'plotless', 'meandering', 'breathes', 'life', 'into', 'the', 'characters', 'such', 'that', 'at', 'film', "'s", 'end', 'you', "'ll", 'feel', 'like', 'an', 'old', 'friend', 'of', 'the', 'family.<br', '/><br', '/>A', 'wonderfully', 'crafted', 'character', '-', 'driven', 'film', 'from', 'the', 'director', 'of', '"', 'Driving', 'Miss', 'Daisy', '"', ',', '"', 'RIL', '"', 'is', 'a', 'somewhat', 'obscure', 'little', '"', 'sleeper', '"', 'which', 'will', 'appeal', 'most', 'to', 'mature', 'audiences', '.'], 'label'

In [3]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [4]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Do not use BucketIterator in your implementation because you are required to implement the padding and masking yourself.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data), batch_size=1, device=device)

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
[('the', 202292), (',', 192366), ('.', 166236), ('a', 109737), ('and', 109602), ('of', 100866), ('to', 93857), ('is', 76621), ('in', 61539), ('I', 54351), ('it', 53589), ('that', 49151), ('"', 43941), ("'s", 43540), ('this', 42430), ('-', 37794), ('/><br', 35757), ('was', 34791), ('as', 30696), ('with', 30187)]
['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']
defaultdict(None, {'neg': 0, 'pos': 1})


In [168]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LR(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.fc = nn.Linear(embedding_dim, output_dim)
    def forward(self, text):
        embedded = self.embedding(text).squeeze().sum(0)
        return self.fc(embedded)
    
# define LSTM Classifier model class
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, embedding_dim, output_dim, hidden_dim):
        super().__init__()
        # the word vector for index 0 will be set to zero
        self.embedding = nn.Embedding(
            input_dim, embedding_dim, padding_idx=0
        )
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(
            hidden_dim, output_dim
        )

    def forward(self, text, lens):

        embedded = self.embedding(text)
        embedded = pack_padded_sequence(
            embedded, lens, enforce_sorted=False
        )
        out, h_n = self.lstm(embedded)
        out, seq_lengths = pad_packed_sequence(
            out
        )
        
        out = torch.stack(
            [torch.mean(out[:length.item(), i, :], 0) for i, length in enumerate(lens)]
        )

        return self.fc(out).squeeze(1)

In [169]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1

lr_model = LR(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM)

In [220]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
HIDDEN_DIM = 4

lstm_model = LSTMClassifier(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, HIDDEN_DIM)

In [221]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The LR model has {count_parameters(lr_model):,} trainable parameters')
print(f'The LSTM model has {count_parameters(lstm_model):,} trainable parameters')

The LR model has 2,500,301 trainable parameters
The LSTM model has 2,501,901 trainable parameters


In [222]:
import torch.optim as optim

lr_optimizer = optim.SGD(lr_model.parameters(), lr=1e-3)
lstm_optimizer = optim.SGD(lstm_model.parameters(), lr=1e-3)

In [223]:
lr_criterion = nn.BCEWithLogitsLoss()
lstm_criterion = nn.BCEWithLogitsLoss()

In [224]:
lr_model = lr_model.to(device)
lr_criterion = lr_criterion.to(device)

lstm_model = lstm_model.to(device)
lstm_criterion = lstm_criterion.to(device)

In [225]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [226]:
from tqdm import tqdm
def train(model, iterator, optimizer, criterion, is_lstm=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for instance in tqdm(iterator, desc="Training...", total=len(iterator)):
        
        optimizer.zero_grad()
        
        predictions = []
        if is_lstm:
            predictions = model(instance.text, torch.Tensor([len(instance.text)]).int())
        else:
            predictions = model(instance.text)
        
        loss = criterion(predictions, instance.label)
        
        acc = binary_accuracy(predictions, instance.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [227]:
def evaluate(model, iterator, criterion, is_lstm=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for instance in tqdm(iterator, desc="Evaluating...", total=len(iterator)):
        
            predictions = []
            if is_lstm:
                predictions = model(instance.text, torch.Tensor([len(instance.text)]).int())
            else:
                predictions = model(instance.text)
            
            loss = criterion(predictions, instance.label)
            
            acc = binary_accuracy(predictions, instance.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [228]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(lr_model, train_iterator, lr_optimizer, lr_criterion)
    valid_loss, valid_acc = evaluate(lr_model, valid_iterator, lr_criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'lr-starter-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [229]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(lstm_model, train_iterator, lstm_optimizer, lstm_criterion, is_lstm=True)
    valid_loss, valid_acc = evaluate(lstm_model, valid_iterator, lstm_criterion, is_lstm=True)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(lstm_model.state_dict(), 'lstm-starter-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Training...:   0%|          | 3/17500 [00:00<10:13, 28.50it/s]

Epoch: 01 | Epoch Time: 15m 24s
	Train Loss: 0.694 | Train Acc: 51.94%
	 Val. Loss: 0.692 |  Val. Acc: 53.49%


Training...:   0%|          | 2/17500 [00:00<15:27, 18.88it/s]

Epoch: 02 | Epoch Time: 15m 35s
	Train Loss: 0.692 | Train Acc: 53.89%
	 Val. Loss: 0.692 |  Val. Acc: 53.93%


Training...:   1%|          | 151/17500 [00:08<15:22, 18.81it/s]


KeyboardInterrupt: 

In [230]:
lstm_model.load_state_dict(torch.load('lstm-starter-model.pt'))

test_loss, test_acc = evaluate(lstm_model, test_iterator, lstm_criterion, is_lstm=True)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Evaluating...: 100%|██████████| 25000/25000 [03:18<00:00, 125.87it/s]

Test Loss: 0.692 | Test Acc: 54.38%


In [ ]:
# Test of model correctness
max_n_test_instances = 5
i = 1
for instance in valid_iterator:
  score = model(instance.text)
  print(score)
  if i >= max_n_test_instances:
    break
  else:
    i += 1


tensor([-6.7634], device='cuda:0', grad_fn=<AddBackward0>)
tensor([0.5967], device='cuda:0', grad_fn=<AddBackward0>)
tensor([5.0447], device='cuda:0', grad_fn=<AddBackward0>)
tensor([-0.4295], device='cuda:0', grad_fn=<AddBackward0>)
tensor([7.1855], device='cuda:0', grad_fn=<AddBackward0>)
